**Анализ тональности (сентимент-анализ)** - важный инструмент как индустриальных, так и научных исследований. Вы можете с его помощью оценить как рецепцию нововведений или рекламной кампании, так и отношение, например, к Наполеону Бонапарту в широком корпусе текстов. 
Существует несколько подходов к оценке тональности в тексте:
1. Основанный на лексиконе, т.е. таких списках слов, которые заранее размечены как, например, "позитивный" или "негативный";
2. Классическое машинное обучение: можно построить вектора заранее размеченных текстов, а зачем настроить классификатор (подобно тому, как мы работали со спамом);
3. Методы глубокого обучения, например, с помощью BERT.

Сами подходы к анализу текста тоже могут быть разнообразными: мы можем оценивать текст целиком, а может дробить его на части, исследуя искомый психологизм (например, нагнетание негатива в романе); мы можем находить отдельные сущности или концепции (уже упомянутый Наполеон или разделение на оценку сервиса или еды в отзывах на ресторан).

(Вообще, конечно, это умеют делать современные генеративные модели до определенной степени, но мы все равно научимся основным способам, не подключающим их; тем более что обработка большого массива текстов с их помощью потребует либо API, либо разворачивания своей модели, либо весьма заумного промптинга и немалых усилий). Так оценивает следующий текст Grok:  
*Он злобно приподнялся, чувствуя, что весь разбит; кости его болели. На дворе совершенно густой туман и ничего разглядеть нельзя. Час пятый в исходе; проспал! Он встал и надел свою жакетку и пальто, еще сырые. Нащупав в кармане револьвер, он вынул его и поправил капсюль; потом сел, вынул из кармана записную книжку и на заглавном, самом заметном листке, написал крупно несколько строк. Перечитав их, он задумался, облокотясь на стол. Револьвер и записная книжка лежали тут же, у локтя. Проснувшиеся мухи лепились на нетронутую порцию телятины, стоявшую тут же на столе. Он долго смотрел на них и, наконец, свободною правою рукой начал ловить одну муху. Долго истощался он в усилиях, но никак не мог поймать. Наконец, поймав себя на этом интересном занятии, очнулся, вздрогнул, встал и решительно пошел из комнаты. Через минуту он был на улице.*

Тональность текста **негативная**.  

Обоснование: Текст описывает мрачные, тягостные эмоции и физическое состояние героя (злость, боль, разбитость), а также создает атмосферу безысходности и подавленности через упоминания густого тумана, сырости, револьвера и записной книжки с крупно написанными строками, что намекает на возможные мрачные намерения. Действия героя, такие как бесцельная попытка ловить муху и его внезапное "очнулся, вздрогнул", усиливают ощущение внутренней тревоги и отчаяния.

Однако начнем по порядку. Основанный на **лексиконе** подход прост в применении и не требует мощных вычислений, требует только словаря, вполне уместен в употреблении, хотя и имеет свои недостатки вроде неумения ухватить контекст (например, иронию).

Основная проблема, конечно, лексикон. Как его собрать? Можно, например, достать какой-то небольшой набор из word2vec:

In [ ]:
import gensim
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer
import matplotlib.pyplot as plt
from collections import Counter
morph = MorphAnalyzer()
stopwords_ru = stopwords.words('russian')

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

Соберем все похожие слова на "негативный":

In [ ]:
neg_adj_sim = model.most_similar('негативный_ADJ', topn=100)
neg_adj = [sim for sim in neg_adj_sim if sim[0].endswith('ADJ') and sim[1] > 0.42]
neg_adj

Выглядит даже неплохо! Кроме того, что там есть несколько неудачных слов (*позитивный, положительный*), которые мы можем элиминировать: 

In [ ]:
neg_adj_fin = [adj[0][:-4] for adj in neg_adj if adj[0][:-4] not in ['позитивный', 'положительный']]
print(neg_adj_fin)
print(len(neg_adj_fin))

Проделаем то же самое с позитивным:

In [ ]:
pos_adj_sim = model.most_similar('позитивный_ADJ', topn=100)
pos_adj = [sim for sim in pos_adj_sim if sim[0].endswith('ADJ') and sim[1] > 0.42]
pos_adj

Здесь явно лишние: негативный, отрицательный, субъективный, пессимистический, когнитивный, неоднозначный, поведенческий, скептический, аффективный, амбивалентный, социальный, психологический, деструктивный, специфический, антисоциальный, гедонистический, противоречивйы, манипулятивный, ценностный, коммуникативный, эмпирический, институциональный.

In [ ]:
pos_adj_fin = [adj[0][:-4] for adj in pos_adj if adj[0][:-4] not in ['негативный', 'отрицательный', 'субъективный', 'пессимистический', 'когнитивный', 'неоднозначный', 'поведенческий', 'скептический', 'аффективный', 'амбивалентный', 'социальный', 'психологический', 'деструктивный', 'специфический', 'антисоциальный', 'гедонистический', 'противоречивый', 'манипулятивный', 'ценностный', 'коммуникативный', 'эмпирический', 'институциональный']]
print(pos_adj_fin)
print(len(pos_adj_fin))

Сделаем это с существительными:

In [ ]:
neg_noun_sim = model.most_similar('горе_NOUN', topn=100)
neg_noun = [sim for sim in neg_noun_sim if sim[0].endswith('NOUN') and sim[1] > 0.5]
neg_noun 

Уберем слова радость, утешение, сердце.

In [ ]:
neg_noun_fin = [noun[0][:-5] for noun in neg_noun if noun[0][:-5] not in ['радость', 'утешение', 'сердце']]
print(neg_noun_fin)
print(len(neg_noun_fin))

In [ ]:
pos_noun_sim = model.most_similar('радость_NOUN', topn=100)
pos_noun = [sim for sim in pos_noun_sim if sim[0].endswith('NOUN') and sim[1] > 0.5]
#print(pos_noun) #лишние явно печаль, горе, горесть, грусть, скорбь, страдание, тоска, скорба, горечь, огорчение, разочарование, страх, отчаяние
pos_noun_fin = [noun[0][:-5] for noun in pos_noun if noun[0][:-5] not in ['печаль', 'горе', 'горесть', 'грусть', 'скорбь', 'страдание', 'тоска', 'скорба', 'горечь', 'огорчение', 'разочарование', 'страх', 'отчаяние']]
print(pos_noun_fin)
print(len(pos_noun_fin))

И наречия:

In [ ]:
pos_adv_sim = model.most_similar('позитивно_ADV', topn=100)
pos_adv = [sim for sim in pos_adv_sim if sim[0].endswith('ADV') and sim[1] > 0.5]
#print(pos_adv) #неоднозначно, критично, негативно, критически, отрицательно, прохладно, нелестно, сдержанно, скептически, пессимистически, противоречиво, неодобрительно, недоброжелательно, нелицеприятно, пренебрежительно, неблагоприятно, недружелюбно, настороженно, своеобразно, трезво
pos_adv_fin = [adv[0][:-4] for adv in pos_adv if adv[0][:-4] not in ['неоднозначно', 'критично', 'негативно', 'критически', 'отрицательно', 'прохладно', 'нелестно', 'сдержанно', 'скептически', 'пессимистически', 'противоречиво', 'неодобрительно', 'недоброжелательно', 'нелицеприятно', 'пренебрежительно', 'неблагоприятно', 'недружелюбно', 'настороженно', 'своеобразно']]
print(pos_adv_fin)
print(len(pos_adv_fin))

In [ ]:
neg_adv_sim = model.most_similar('негативно_ADV', topn=100)
neg_adv = [sim for sim in neg_adv_sim if sim[0].endswith('ADV') and sim[1] > 0.5]
#print(neg_adv) #положительно, позитивно, благоприятно, благожелательно, сочувственно, нейтрально, лояльно, благосклонно, оптимистически, доброжелательно, оптимистично, положительный, благотворно, серьезно, серьёзно
neg_adv_fin = [adv[0][:-4] for adv in neg_adv if adv[0][:-4] not in ['положительно', 'позитивно', 'благоприятно', 'благожелательно', 'сочувственно', 'нейтрально', 'лояльно', 'благосклонно', 'оптимистически', 'доброжелательно', 'оптимистично', 'положительный', 'благотворно', 'серьезно', 'серьёзно']]
print(neg_adv_fin)
print(len(neg_adv_fin))

Этого достаточно! (для серьезной работы нет). В целом посидев один вечер можно собрать вполне презентабельный лексикон на несколько тысяч слов. Сейчас у нас получается следующее:

In [ ]:
positive = pos_adj_fin + pos_adv_fin + pos_noun_fin
print(len(positive))
negative = neg_adj_fin + neg_adv_fin + neg_noun_fin
print(len(negative))

Ну что ж. И с меньшим в бой бросались! При этом это не так уж плохо, благодаря векторам можно, приложив некоторые усилия, собрать нюансированный лексикон, подстроенный под определенные отдельные задачи разметки тональности. Посмотрим, что получится на Евгении Онегине.

In [ ]:
with open('EugeneOnegin.txt', encoding='utf-8') as txt:
    text_orig = txt.read()
    corpus = re.split(r'ГЛАВА \w+\b', text_orig)
    print(len(corpus))
    clean_texts = []
    just_texts = []
    for text in corpus:
        text = re.sub(r'\n', ' ', text)
        just_texts.append(text)
        text = re.sub('[^а-яА-ЯёЁ -]', '', text.lower())
        lemmatized_text = [morph.parse(tok)[0].normal_form for tok in word_tokenize(text)]
        text_no_stop = [token for token in lemmatized_text if token not in stopwords_ru]
        clean_texts.append(text_no_stop)
        
print(len(clean_texts))

Напишем простенький скорер:

In [ ]:
neg_scores = []
pos_scores = []
for text in clean_texts:
    neg_score = 0
    pos_score = 0
    len_t = len(text)
    for token in text:
        if token in positive:
            pos_score += 1
        elif token in negative:
            neg_score += 1
    neg_scores.append(round(neg_score/len_t*100, 2))
    pos_scores.append(round(pos_score/len_t*100, 2))

print(neg_scores)
print(pos_scores)

In [ ]:
def corpus_scorer(corpus):
    neg_scores = []
    pos_scores = []
    for text in corpus:
        neg_score = 0
        pos_score = 0
        len_t = len(text)
        for token in text:
            if token in positive:
               pos_score += 1
            elif token in negative:
                neg_score += 1
        neg_scores.append(round(neg_score/len_t*100, 2))
        pos_scores.append(round(pos_score/len_t*100, 2))

    return {'pos_score': pos_scores, 'neg_score': neg_scores}

In [ ]:
scores = corpus_scorer(clean_texts)
print(scores)

Как водится, нанесем на график:

In [ ]:
chapters = list(range(1, len(clean_texts) + 1))
plt.figure(figsize=(10, 6))
plt.plot(chapters, scores['pos_score'], label='Позитив', color='skyblue', marker='o', linewidth=2)
plt.plot(chapters, scores['neg_score'], label='Негатив', color='red', marker='s', linewidth=2)
plt.title('Тональность по главам')
plt.xlabel('Глава')
plt.ylabel('Оценка тональности')
plt.legend()

Все это, конечно, ерунда, потому что лексикон очень маленький. Можно попробовать модель сгенерировать лексиконы.

In [ ]:
pos_verbs = ['радовать', 'вдохновлять', 'поддерживать', 'созидать', 'улыбаться', 'любить', 'восхищать', 'помогать', 'воодушевлять', 'творить', 'украшать', 'улучшать', 'развивать', 'согревать', 'заботиться', 'благодарить', 'веселить', 'обнимать', 'дарить', 'делиться', 'светить', 'сиять', 'благоухать', 'процветать', 'гармонизировать', 'укреплять', 'обогащать', 'оживлять', 'возвышать', 'восстанавливать', 'исцелять', 'успокаивать', 'освещать', 'наслаждаться', 'восхищаться', 'мечтать', 'стремиться', 'достигать', 'побеждать', 'преуспевать', 'радоваться', 'ликовать', 'благословлять', 'лелеять', 'хвалить', 'поощрять', 'мотивировать', 'вдохновляться', 'доверять', 'содействовать', 'сближать', 'объединять', 'сплачивать', 'создавать', 'совершенствовать', 'обновлять', 'оживить', 'праздновать', 'улыбать', 'щебетать', 'петь', 'танцевать', 'смеяться', 'шутить', 'играть', 'раскрывать', 'открывать', 'изучать', 'познавать', 'расти', 'цвести', 'усиливать', 'возрождать', 'освежать', 'бодрить', 'тонизировать', 'зажигать', 'вспыхивать', 'сияять', 'блестеть', 'сверкать', 'украшать', 'преображать', 'улучшать', 'вознаграждать', 'ценивать', 'уважать', 'баловать', 'ласкать', 'умилять', 'трогать', 'вдохновлять', 'взлетать', 'парить', 'свободствовать', 'освобождать', 'распускаться', 'расцветать', 'умиротворять', 'согревать', 'защищать']
pos_nouns = ['радость', 'счастье', 'любовь', 'доброта', 'успех', 'гармония', 'красота', 'вдохновение', 'тепло', 'свет', 'улыбка', 'дружба', 'мечта', 'надежда', 'сила', 'энергия', 'свобода', 'мир', 'спокойствие', 'блаженство', 'восторг', 'ликование', 'доверие', 'забота', 'нежность', 'ласковость', 'чудо', 'волшебство', 'благодать', 'процветание', 'изобилие', 'творчество', 'совершенство', 'победа', 'триумф', 'гордость', 'честь', 'достоинство', 'уважение', 'благодарность', 'веселье', 'праздник', 'смех', 'игра', 'яркость', 'сияние', 'блеск', 'цвет', 'аромат', 'свежесть', 'жизнь', 'здоровье', 'бодрость', 'энтузиазм', 'оптимизм', 'мотивация', 'цель', 'достижение', 'развитие', 'рост', 'процветание', 'благополучие', 'комфорт', 'уют', 'тепло', 'гостеприимство', 'дружелюбие', 'единство', 'солидарность', 'взаимопонимание', 'поддержка', 'щедрость', 'великодушие', 'терпение', 'мудрость', 'проницательность', 'честность', 'искренность', 'открытость', 'правда', 'справедливость', 'гармония', 'баланс', 'умиротворение', 'безмятежность', 'чистота', 'ясность', 'светозарность', 'возвышенность', 'поэзия', 'музыка', 'танец', 'искра', 'звезда', 'солнце', 'радуга', 'цветок', 'весна', 'жизнерадостность', 'обаяние', 'харизма']
pos_adjs = ['радостный', 'счастливый', 'добрый', 'прекрасный', 'вдохновляющий', 'теплый', 'светлый', 'чудесный', 'великолепный', 'удивительный', 'замечательный', 'гармоничный', 'спокойный', 'уверенный', 'яркий', 'душевный', 'искренний', 'дружелюбный', 'восторженный', 'уважительный', 'похвальный', 'трогательный', 'ласковый', 'милый', 'обаятельный', 'энергичный', 'жизнерадостный', 'успешный', 'продуктивный', 'творческий', 'изящный', 'красивый', 'элегантный', 'грациозный', 'мудрый', 'глубокий', 'проницательный', 'честный', 'открытый', 'щедрый', 'великодушный', 'терпеливый', 'справедливый', 'деликатный', 'тактичный', 'внимательный', 'заботливый', 'любовный', 'нежный', 'умиротворенный', 'светлый', 'ясный', 'чистосердечный', 'благородный', 'достойный', 'гордый', 'смелый', 'решительный', 'надежный', 'стабильный', 'безопасный', 'уютный', 'комфортный', 'приятный', 'аппетитный', 'вкусный', 'ароматный', 'свежий', 'бодрый', 'здоровый', 'сильный', 'мощный', 'динамичный', 'увлекательный', 'захватывающий', 'волшебный', 'фантастический', 'необыкновенный', 'оригинальный', 'креативный', 'инновационный', 'прогрессивный', 'эффективный', 'результативный', 'блаженный', 'ликующий', 'восхитительный', 'бесподобный', 'совершенный', 'идеальный', 'возвышенный', 'поэтичный', 'живописный', 'радужный', 'безмятежный', 'умиротворяющий', 'свободный', 'непринужденный', 'естественный', 'плавный', 'утонченный']
pos_advs = ['радостно', 'счастливо', 'доброжелательно', 'прекрасно', 'вдохновенно', 'тепло', 'светло', 'чудесно', 'великолепно', 'удивительно', 'замечательно', 'гармонично', 'спокойно', 'уверенно', 'ярко', 'душевно', 'искренне', 'дружелюбно', 'восторженно', 'уважительно', 'похвально', 'трогательно', 'ласково', 'мило', 'обаятельно', 'энергично', 'жизнерадостно', 'успешно', 'продуктивно', 'творчески', 'изящно', 'красиво', 'элегантно', 'грациозно', 'мудро', 'глубоко', 'проницательно', 'честно', 'открыто', 'щедро', 'великодушно', 'терпеливо', 'справедливо', 'деликатно', 'тактично', 'внимательно', 'заботливо', 'любовно', 'нежно', 'умиротворенно', 'ясно', 'чистосердечно', 'благородно', 'достойно', 'гордо', 'смело', 'решительно', 'надежно', 'стабильно', 'безопасно', 'уютно', 'комфортно', 'приятно', 'аппетитно', 'вкусно', 'ароматно', 'свежо', 'бодро', 'здорово', 'сильно', 'мощно', 'динамично', 'увлекательно', 'захватывающе', 'волшебно', 'фантастически', 'необыкновенно', 'оригинально', 'креативно', 'инновационно', 'прогрессивно', 'эффективно', 'результативно', 'блаженно', 'ликующе', 'восхитительно', 'бесподобно', 'совершенно', 'идеально', 'возвышенно', 'поэтично', 'живописно', 'радужно', 'безмятежно', 'умиротворяюще', 'свободно', 'непринужденно', 'естественно', 'плавно', 'утонченно']
positives = pos_verbs + pos_nouns + pos_adjs + pos_advs
neg_verbs = ['разрушать', 'вредить', 'обманывать', 'оскорблять', 'унижать', 'злиться', 'гневаться', 'кричать', 'ругаться', 'обвинять', 'игнорировать', 'отвергать', 'презирать', 'ненавидеть', 'завидовать', 'мстить', 'угрожать', 'давить', 'подавлять', 'издеваться', 'насмехаться', 'провоцировать', 'конфликтовать', 'ссориться', 'спорить', 'возражать', 'отрицать', 'саботировать', 'мешать', 'затруднять', 'тормозить', 'ограничивать', 'запрещать', 'нарушать', 'портить', 'ломать', 'губить', 'разрушать', 'уничтожать', 'разворовывать', 'красть', 'обворовывать', 'мошенничать', 'лгать', 'фальсифицировать', 'подделывать', 'искажать', 'преувеличивать', 'умалчивать', 'скрывать', 'игнорировать', 'пренебрегать', 'отказывать', 'отталкивать', 'изгонять', 'высмеивать', 'критиковать', 'осуждать', 'порицать', 'упрекать', 'жаловаться', 'ныть', 'ворчать', 'брюзжать', 'раздражать', 'злословить', 'сплетничать', 'клеветать', 'очернять', 'подозревать', 'сомневаться', 'недооценивать', 'переоценивать', 'обесценивать', 'умалять', 'принижать', 'дискриминировать', 'ущемлять', 'ограблять', 'нападать', 'агрессировать', 'драться', 'бить', 'ранить', 'мучить', 'терзать', 'изматывать', 'истощать', 'эксплуатировать', 'шантажировать', 'манипулировать', 'запугивать', 'давить', 'принуждать', 'навязывать', 'обременять', 'засорять', 'загрязнять', 'отравлять', 'разочаровывать', 'подводить']
neg_nouns = ['горе', 'печаль', 'страдание', 'тоска', 'грусть', 'скорбь', 'разочарование', 'страх', 'отчаяние', 'гнев', 'злость', 'ненависть', 'зависть', 'месть', 'конфликт', 'ссора', 'обида', 'оскорбление', 'унижение', 'презрение', 'ложь', 'обман', 'предательство', 'измена', 'клевета', 'сплетня', 'злословие', 'насмешка', 'издевательство', 'провокация', 'угроза', 'давление', 'подавление', 'агрессия', 'насилие', 'жестокость', 'мучение', 'боль', 'рана', 'потеря', 'ущерб', 'разрушение', 'хаос', 'беспорядок', 'кризис', 'паника', 'тревога', 'сомнение', 'неуверенность', 'недоверие', 'подозрение', 'злоба', 'раздражение', 'досада', 'огорчение', 'горечь', 'разрушение', 'упадок', 'деградация', 'провал', 'неудача', 'крах', 'банкротство', 'бедствие', 'катастрофа', 'трагедия', 'несчастье', 'беда', 'проблема', 'трудность', 'препятствие', 'ограничение', 'запрет', 'нарушение', 'воровство', 'грабеж', 'мошенничество', 'фальшь', 'подделка', 'искажение', 'ошибка', 'недостаток', 'дефект', 'порок', 'зависимость', 'эксплуатация', 'шантаж', 'манипуляция', 'принуждение', 'дискриминация', 'ущемление', 'несправедливость', 'жесткость', 'равнодушие', 'холодность', 'отчуждение', 'одиночество', 'изоляция', 'безнадежность', 'апатия', 'скука', 'усталость']
neg_adjs = ['негативный', 'отрицательный', 'пессимистический', 'скептический', 'деструктивный', 'агрессивный', 'злой', 'гневный', 'враждебный', 'жестокий', 'холодный', 'равнодушный', 'безразличный', 'надменный', 'высокомерный', 'презрительный', 'насмешливый', 'саркастический', 'циничный', 'обидный', 'оскорбительный', 'унизительный', 'грубый', 'хамский', 'вульгарный', 'неприятный', 'отталкивающий', 'отвратительный', 'омерзительный', 'тревожный', 'страшный', 'пугающий', 'мрачный', 'угрюмый', 'печальный', 'грустный', 'тоскливый', 'унылый', 'безрадостный', 'безнадежный', 'отчаянный', 'трагичный', 'горестный', 'болезненный', 'мучительный', 'терзающий', 'изнурительный', 'утомительный', 'скучный', 'монотонный', 'лживый', 'обманный', 'коварный', 'предательский', 'вероломный', 'неискренний', 'фальшивый', 'лицемерный', 'подлый', 'низкий', 'бесчестный', 'непорядочный', 'аморальный', 'безнравственный', 'жесткий', 'суровый', 'неумолимый', 'несправедливый', 'пристрастный', 'предвзятый', 'злопамятный', 'мстительный', 'завистливый', 'жадный', 'скупой', 'эгоистичный', 'корыстный', 'манипулятивный', 'хитрый', 'лукавый', 'скрытный', 'подозрительный', 'недоверчивый', 'трусливый', 'слабый', 'беспомощный', 'ненадежный', 'нестабильный', 'хаотичный', 'беспорядочный', 'разрушительный', 'вредный', 'токсичный', 'опасный', 'угрожающий', 'неблагоприятный', 'неудачный', 'провальный', 'дефектный', 'некачественный', 'неполноценный']
neg_advs = ['эгоистично', 'шумно', 'грубо', 'жадно', 'лживо', 'лениво', 'нагло', 'злобно', 'трусливо', 'подло', 'мелочно', 'завистливо', 'высокомерно', 'глупо', 'раздражительно', 'безответственно', 'коварно', 'надменно', 'мерзко', 'лицемерно', 'бестактно', 'вульгарно', 'сварливо', 'мрачно', 'назойливо', 'корыстно', 'апатично', 'безнравственно', 'бездарно', 'бездушно', 'жестоко', 'противно', 'цинично', 'черство', 'упрямо', 'хамовато', 'истерично', 'злопамятно', 'невнимательно', 'неприятно', 'равнодушно', 'тщеславно', 'фальшиво', 'бестолково', 'беспощадно', 'болтливо', 'властно', 'инертно', 'невежественно', 'пугливо', 'агрессивно', 'алчно', 'безалаберно', 'безвольно', 'безрадостно', 'безрассудно', 'беспутно', 'зловеще', 'зловредно', 'болезненно', 'вздорно', 'некомпетентно', 'непослушно', 'непростительно', 'неряшливо', 'несносно', 'нетерпимо', 'нудно', 'обидчиво', 'опасно', 'отвратительно', 'пассивно', 'печально', 'плоско', 'похотливо', 'презрительно', 'пренебрежительно', 'придирающе', 'разрушительно', 'самовлюблённо', 'саркастично', 'свирепо', 'скучно', 'слабо', 'слизко', 'строптиво', 'суеверно', 'угрюмо', 'хаотично', 'холодно', 'негодно', 'недобро', 'недоверчиво', 'нежизнеспособно', 'накладно', 'неверно', 'самонадеянно', 'язвительно', 'безнадёжно', 'уныло']
negatives = neg_verbs + neg_nouns + neg_adjs + neg_advs

In [ ]:
def corpus_scorer_ed(corpus):
    neg_scores = []
    pos_scores = []
    diff_scores = []
    for text in corpus:
        neg_score = 0
        pos_score = 0
        len_t = len(text)
        for token in text:
            if token in positives:
               pos_score += 1
            elif token in negatives:
                neg_score += 1
        neg_scores.append(round(neg_score/len_t*100, 2))
        pos_scores.append(round(pos_score/len_t*100, 2))
        diff_scores.append(round(pos_score/len_t*100 - neg_score/len_t*100, 2)) #добавим разницу
    
    return {'pos_score': pos_scores, 'neg_score': neg_scores, 'pos_neg': diff_scores}

In [ ]:
scores = corpus_scorer_ed(clean_texts)
scores

С новым списком долги негативных и позитивных явно выросли. При этом в целом Евгений Онегин остается все еще более позитивным текстом.

In [ ]:
chapters = list(range(1, len(clean_texts) + 1))
plt.figure(figsize=(10, 6))
plt.plot(chapters, scores['pos_score'], label='Позитив', color='skyblue', marker='o', linewidth=2)
plt.plot(chapters, scores['neg_score'], label='Негатив', color='red', marker='s', linewidth=2)
plt.plot(chapters, scores['pos_neg'], label='Позитив-негатив', color='green', marker='^', linewidth=2)
plt.title('Тональность по главам')
plt.xlabel('Глава')
plt.ylabel('Оценка тональности')
plt.legend()

Можно интерпретировать данные в графиках по-другому. Например, показать долю эмоциональных слов в целом, а не разницу (позже добавим в наш скорер):

In [ ]:
emotions = []
for index in range(len(scores['pos_score'])):
    #print(scores['pos_score'][index] + scores['neg_score'][index])
    emotions.append(scores['pos_score'][index] + scores['neg_score'][index])
print(emotions)

#то же самое
emotions = [scores['pos_score'][index] + scores['neg_score'][index] for index in range(len(scores['pos_score']))]
print(emotions)

In [ ]:
def corpus_scorer_ed(corpus):
    neg_scores = []
    pos_scores = []
    diff_scores = []
    num_tokens = []
    num_pos = []
    num_neg = []
    for text in corpus:
        neg_score = 0
        pos_score = 0
        len_t = len(text)
        for token in text:
            if token in positives:
               pos_score += 1
            elif token in negatives:
                neg_score += 1
        neg_scores.append(round(neg_score/len_t*100, 2))
        pos_scores.append(round(pos_score/len_t*100, 2))
        diff_scores.append(round(pos_score/len_t*100 - neg_score/len_t*100, 2)) #добавим разницу
        num_tokens.append(len_t)
        num_pos.append(pos_score)
        num_neg.append(neg_score)
    return {'pos_score': pos_scores, 'neg_score': neg_scores, 'pos_neg': diff_scores,
           'num_tokens': num_tokens, 'num_pos': num_pos, 'num_neg': num_neg}

In [ ]:
scores = corpus_scorer_ed(clean_texts)
scores

In [ ]:
chapters = list(range(1, len(clean_texts) + 1))
plt.figure(figsize=(10, 6))
plt.plot(chapters, emotions, label='Эмоциональные слова', color='skyblue', marker='o', linewidth=2)
plt.title('Эмоциональность по главам, в %')
plt.xlabel('Глава')
plt.ylabel('Оценка тональности')
plt.legend()

Отобразим сумму эмоциональных слов на фоне всех слов:

In [ ]:
emotions_sum = [scores['num_pos'][index] + scores['num_neg'][index] for index in range(len(scores['pos_score']))]
print(emotions_sum)

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(chapters, scores['num_tokens'], color='gray', label='Всего слов', alpha=0.3)
plt.bar(chapters, emotions_sum, color='red', label='Всего эмоциональных слов', alpha=0.3)
plt.xlabel('Глава')
plt.ylabel('Количество слов')
plt.title('Драматичный график суммы слов')
plt.legend()

Ну и наконец готовые лексиконы. Они обычно намного крупнее и мощнее тех игрушек, которые мы попробовали сделать сами.

Воспользуемся русентилексом: https://www.labinform.ru/pub/rusentilex/

!Словарь РуСентиЛекс  
! Структура:   
! 1. слово или словосочетание,  
! 2. Часть речи или синтаксический тип группы,  
! 3. слово или словосочетание в лемматизированной форме,   
! 4. Тональность: позитивная (positive), негативная(negative), нейтральная (neutral) или неопределеная   оценка, зависит от контекста (positive/negative),  
! 5. Источник: оценка (opinion), чувство (feeling), факт (fact),  
! 6. Если тональность отличается для разных значений многозначного слова, то перечисляются все значения слова по тезаурусу РуТез и дается отсылка на сооветствующее понятие - имя понятия в кавычках.
!  
!RuSentiLex Structure  
!1. word or phrase,  
!2. part of speech or type of syntactic group,  
!3. initial word (phrase) in a lemmatized form,  
!4. Sentiment: positive, negative, neutral or positive/negative (indefinite, depends on the context),  
!5. Source: opinion, feeling (private state), or fact (sentiment connotation),  
!6. Ambiguity: if sentiment is different for senses of an ambiguous word, then sentiment orientations for all senses are described, the senses  
!are labeled with the RuThes concept names.  

In [ ]:
with open('rusentilex_2017.txt', encoding='utf-8') as txt:
    text = txt.read()
    print(text[:200])

In [ ]:
lines = text.split('\n')
lines[:10]

In [ ]:
pos_lex = []
for line in lines:
    line_s = line.split(',')
    try:
        if line_s[3].strip() == 'positive':
            pos_lex.append(line_s[0])
    except IndexError:
        continue
print(pos_lex[:10])
print(len(pos_lex))

In [ ]:
neg_lex = []
for line in lines:
    line_s = line.split(',')
    try:
        if line_s[3].strip() == 'negative':
            neg_lex.append(line_s[0])
    except IndexError:
        continue
print(neg_lex[:10])
print(len(neg_lex))

 Немного изменим скорер:

In [ ]:
def corpus_scorer_ed(corpus):
    neg_scores = []
    pos_scores = []
    diff_scores = []
    num_tokens = []
    num_pos = []
    num_neg = []
    for text in corpus:
        neg_score = 0
        pos_score = 0
        len_t = len(text)
        for token in text:
            if token in pos_lex:
               pos_score += 1
            elif token in neg_lex:
                neg_score += 1
        neg_scores.append(round(neg_score/len_t*100, 2)) 
        pos_scores.append(round(pos_score/len_t*100, 2)) 
        diff_scores.append(round(pos_score/len_t*100 - neg_score/len_t*100, 2)) #добавим разницу
        num_tokens.append(len_t)
        num_pos.append(pos_score)
        num_neg.append(neg_score)
    return {'pos_score': pos_scores, 'neg_score': neg_scores, 'pos_neg': diff_scores,
           'num_tokens': num_tokens, 'num_pos': num_pos, 'num_neg': num_neg}

In [ ]:
scores = corpus_scorer_ed(clean_texts)
scores

Кажется, скоры явно прояснились. Мы уже видим падение разницы пос-нег в негативную зону в той части, где происходят убийства, грусть и расставания. Ура!

Нарисуем разные графики снова.

In [ ]:
chapters = list(range(1, len(clean_texts) + 1))
plt.figure(figsize=(10, 6))
plt.plot(chapters, scores['pos_score'], label='Позитив', color='skyblue', marker='o', linewidth=2)
plt.plot(chapters, scores['neg_score'], label='Негатив', color='red', marker='s', linewidth=2)
plt.title('Тональность по главам')
plt.xlabel('Глава')
plt.ylabel('Оценка тональности')
plt.legend()

In [ ]:
chapters = list(range(1, len(clean_texts) + 1))
plt.figure(figsize=(10, 6))
plt.plot(chapters, emotions, label='Эмоциональные слова', color='skyblue', marker='o', linewidth=2)
plt.title('Общая эмоциональность по главам, в %')
plt.xlabel('Глава')
plt.ylabel('Оценка тональности')
plt.legend()

In [ ]:
chapters = list(range(1, len(clean_texts) + 1))
plt.figure(figsize=(10, 6))
plt.plot(chapters, scores['pos_neg'], label='Позитив-негатив', color='green', marker='*', linewidth=2)
plt.title('Превалирование позитивных и негативных слов')
plt.xlabel('Глава')
plt.ylabel('Оценка тональности')
plt.legend()

In [ ]:
emotions_sum = [scores['num_pos'][index] + scores['num_neg'][index] for index in range(len(scores['pos_score']))]
print(emotions_sum)
plt.figure(figsize=(10, 6))
plt.bar(chapters, scores['num_tokens'], color='gray', label='Всего слов', alpha=0.3)
plt.bar(chapters, emotions_sum, color='red', label='Всего эмоциональных слов', alpha=0.3)
plt.xlabel('Глава')
plt.ylabel('Количество слов')
plt.title('Драматичный график суммы слов')
plt.legend()

**Задание 1.** Исследуйте с помощью нашего эмоционального скорера любой роман Достоевского. Видны ли там изменения по главам? Предобработайте роман, если нужно. 

In [ ]:
#ваш код здесь

**Задание 2.** Как поменять обработку лексикона, чтобы она вывела вам не эмоциональные слова, а типы источников: факты, мнения или чувства. Выведите графики для таких типов.

In [ ]:
#ваш код здесь

Давайте еще посмотрим, как описывается Москва в каждой из глав.

In [ ]:
tat = []
for text in clean_texts:
    chap_context = []
    for index in range(len(text)):
        if text[index] == 'татьяна':  
            start = max(0, index - 5) 
            end = min(len(text), index + 5 + 1) 
            context = ' '.join(text[start:end])  
            chap_context.append(context)
    tat.append(' '.join(chap_context))
        
eug = []
for text in clean_texts:
    chap_context = []
    for index in range(len(text)):
        if text[index] == 'евгений':  
            start = max(0, index - 5) 
            end = min(len(text), index + 5 + 1) 
            context = ' '.join(text[start:end])  
            chap_context.append(context)
    eug.append(' '.join(chap_context))

In [ ]:
print(tat)
prunt(eug)

In [ ]:
corpus_scorer_ed(tat)

Немного подлатаем:

In [ ]:
def corpus_scorer_ed_nuance(corpus):
    neg_scores = []
    pos_scores = []
    diff_scores = []
    num_tokens = []
    num_pos = []
    num_neg = []
    for text in corpus:
        neg_score = 0
        pos_score = 0
        len_t = len(word_tokenize(text))
        for token in word_tokenize(text):
            if token in pos_lex:
               pos_score += 1
            elif token in neg_lex:
                neg_score += 1
        if len_t != 0:
            neg_scores.append(round(neg_score/len_t*100, 2)) 
            pos_scores.append(round(pos_score/len_t*100, 2)) 
            diff_scores.append(round(pos_score/len_t*100 - neg_score/len_t*100, 2)) #добавим разницу
            num_tokens.append(len_t)
            num_pos.append(pos_score)
            num_neg.append(neg_score)
        else:
            neg_scores.append(0) 
            pos_scores.append(0) 
            diff_scores.append(0) #добавим разницу
            num_tokens.append(len_t)
            num_pos.append(pos_score)
            num_neg.append(neg_score)
            
    return {'pos_score': pos_scores, 'neg_score': neg_scores, 'pos_neg': diff_scores,
           'num_tokens': num_tokens, 'num_pos': num_pos, 'num_neg': num_neg}

In [ ]:
tat_count = corpus_scorer_ed_nuance(tat)
eug_count = corpus_scorer_ed_nuance(eug)

In [ ]:
chapters = list(range(1, len(clean_texts) + 1))
plt.figure(figsize=(10, 6))
plt.plot(chapters, tat_count['pos_neg'], label='Позитив-негатив Татьяны', color='skyblue', marker='*', linewidth=2)
plt.plot(chapters, eug_count['pos_neg'], label='Позитив-негатив Евгения', color='tomato', marker='s', linewidth=2)

for i, (tat_val, eug_val) in enumerate(zip(tat_count['pos_neg'], eug_count['pos_neg'])):
    plt.text(chapters[i], tat_val + 0.5, f'{tat_val:.2f}', ha='left', va='bottom', fontsize=11, color='skyblue')
    plt.text(chapters[i], eug_val - 0.7, f'{eug_val:.2f}', ha='left', va='top', fontsize=11, color='tomato')

plt.title('Превалирование позитивных и негативных слов')
plt.xlabel('Глава')
plt.ylabel('Оценка тональности')
plt.legend()

**Задание 3**. Посмотрите снова в текст Достоевского. Попробуйте поисследовать тональность вокруг определенного объекта или персонажа (деньги, Раскольников, истина, Бог...).

In [ ]:
#ваш код здесь

Нейронные сети и здесь нам могут помочь. 

Воспользуемся ruBERT. Используем blanchefort/rubert-base-cased-sentiment-rusentiment: https://huggingface.co/blanchefort/rubert-base-cased-sentiment.  

А для английского можно использовать NRClex.

In [ ]:
#не забудьте установить библиотеки, если у вас их нет
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

In [ ]:
print("Виды меток:", model.config.id2label)

In [ ]:
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

Предсказание идет по всем текстам: 

In [ ]:
texts = [
    "Этот фильм просто замечательный!",
    "Мне не понравился этот ресторан.",
    "Сегодня обычный день, ничего особенного."
]
for text in texts:
    print(f"Предсказание для '{text}': {predict(text)}")

Посмотрим на главы:

In [ ]:
clean_texts_j = [' '.join(text) for text in clean_texts]

In [ ]:
chap_count = 1
bert_emotions = []
for text in clean_texts_j:
    prediction = predict(text)
    print(f"Предсказание для главы '{chap_count}': {prediction}")
    bert_emotions.append(predict(text))
    chap_count += 1

print(bert_emotions)

Изменим метки:

In [ ]:
word_emotions = []
for emo in bert_emotions:
    if str(emo) == '[1]':
        word_emotions.append('Позитив')
    elif str(emo) == '[2]':
        word_emotions.append('Негатив')
    else:
        word_emotions.append('Нейтральность')

word_emotions

In [ ]:
chapters = list(range(1, len(clean_texts) + 1))
plt.figure(figsize=(10, 6))
plt.plot(chapters, word_emotions, label='Эмоциональные слова', color='tomato', marker='o', linewidth=2)
plt.title('Общая эмоциональность по главам, оценка')
plt.xlabel('Глава')
plt.ylabel('Оценка тональности')
plt.legend()

Решение по главам. Переделаем в гистограмму: 

In [ ]:
plt.figure(figsize=(10, 6))
#plt.bar(list(Counter(word_emotions).keys()), list(Counter(word_emotions).values()), color=['tomato', 'steelblue', 'crimson'])
bars = plt.bar(list(Counter(word_emotions).keys()), list(Counter(word_emotions).values()), color=['tomato', 'steelblue', 'crimson'])
plt.title('Общая эмоциональность по главам')
plt.legend(bars, list(Counter(word_emotions).keys()), title="Настроение", loc="best")
plt.xlabel('Настроение')
plt.ylabel('Количество')
plt.show()

И круговую:

In [ ]:
plt.figure(figsize=(10, 10))
plt.pie(list(Counter(word_emotions).values()), labels=list(Counter(word_emotions).values()), colors=['tomato', 'steelblue', 'crimson'])
plt.title('Общая эмоциональность по главам')
plt.legend(list(Counter(word_emotions).keys()), title="Настроение", loc="best")
plt.show()

А если не по предобработанным текстам? 

In [ ]:
chap_counter = 1
for text in just_texts:
    prediction = predict(text)
    print(f"Предсказание для главы '{chap_counter}': {prediction}")
    chap_counter += 1

Вгоняет в депрессию.

**Задание 3**. Попробуйте разметить главы в тексте Достоевского и визуализировать.

In [ ]:
#ваш код здесь

**Задание 4.** Перепишите процесс предсказания так, чтобы он размечал по предложениям и выводил среднее арифметическое оценки по главе.